In [13]:
import pandas as pd
import tqdm
import math

# AHV-IV MEMENTO

In [ ]:
df = pd.read_csv("indexing/data/to_upsert/ahv_iv_memento/ahv_iv_de_fr_it_tags_subtopics_llm_EMBED.csv")
df.rename(columns={"text_summary": "summary_embedding"}, inplace=True)
len(df)

In [ ]:
df.head()

In [ ]:
df.to_csv("indexing/data/to_upsert/ahv_iv_memento/ahv_iv_de_fr_it_tags_subtopics_llm_EMBED.csv", index=None)

# EAK - AHV LERNBAUSTEIN 2024

In [ ]:
df = pd.read_csv("indexing/data/to_upsert/AHV_Lernbaustein_2024/AHV_Lernbaustein_2024_tags_llm_EMBED.csv")
df.rename(columns={"text_summary": "summary_embedding"}, inplace=True)
len(df)

In [ ]:
df.head()

In [ ]:
df.to_csv("indexing/data/to_upsert/AHV_Lernbaustein_2024/AHV_Lernbaustein_2024_tags_llm_EMBED.csv", index=None)

# EAK PRAXISLEITFADEN 2024

In [ ]:
df = pd.read_csv("indexing/data/to_upsert/Guide_Pratique_CAF_CFC/guide_pratique_caf_cfc_de_tags_llm.csv")
len(df)

In [ ]:
df.head()

# AKIS

In [ ]:
df = pd.read_csv("indexing/data/zas_eak_copilot/akis/akis_EMBED_2.csv")
len(df)

In [ ]:
df.head()

# EAK ADMIN CH

In [ ]:
df = pd.read_csv("indexing/data/to_upsert/eak_admin_ch/eak_admin_ch_de_fr_tags_NEW.csv")
len(df)

In [ ]:
df.head()

# FEDLEX

In [ ]:
df = pd.read_csv("indexing/data/to_upsert/fedlex/lavs.csv")
len(df)

In [ ]:
df.head()

# OFAS

In [14]:
df = pd.read_csv("../preprocessing/output/ofas.csv")
len(df)

64

In [15]:
df.head()

,url,text,language,tags,subtopics,summary,doctype,organizations,hyq,hyq_declarative
0,https://sozialversicherungen.admin.ch/it/d/690...,"1 Panoramica delle direttive sui contributi, s...",it,ahv_services,"Documenti, Basi_AVS, Direttive_contributi",Il documento fornisce una panoramica delle dir...,context_doc,OFAS,Quali sono le direttive principali trattate ne...,Il documento tratta delle direttive principali...
1,https://sozialversicherungen.admin.ch/it/d/695...,Direttive sull’obbligo assicurativo nell’AVS/A...,it,ahv_services,"Documenti, Basi_AVS, Direttive_contributi",Le Direttive sull'obbligo assicurativo nell'AV...,context_doc,OFAS,Quali sono le modifiche apportate al supplemen...,Il supplemento 17 delle direttive sull'obbligo...
2,https://sozialversicherungen.admin.ch/it/d/694...,Direttive sul salario determinante nell ’AVS/A...,it,ahv_services,"Documenti, Basi_AVS, Direttive_contributi",Le Direttive sul salario determinante nell'AVS...,context_doc,OFAS,Quali sono le modifiche apportate alle diretti...,Le modifiche alle direttive sul salario determ...
3,https://sozialversicherungen.admin.ch/it/d/692...,Direttive sulla riscossione dei contributi nel...,it,ahv_services,"Documenti, Basi_AVS, Direttive_contributi",Le Direttive sulla riscossione dei contributi ...,context_doc,OFAS,Quali sono le condizioni per la riscossione de...,Le condizioni per la riscossione dei contribut...
4,https://sozialversicherungen.admin.ch/it/d/692...,", 6S.413/2004 , con- sid. 7.2) Il mancato pag...",it,ahv_services,"Documenti, Basi_AVS, Direttive_contributi",Il testo tratta delle sanzioni previste per i ...,context_doc,OFAS,Quali sono le sanzioni per i datori di lavoro ...,Le sanzioni per i datori di lavoro che non si ...


# AUTOCOMPLETE

In [ ]:
df = pd.read_csv("indexing/data/to_upsert/autocomplete/question.csv")

df["text_embedding"] = None

len(df)

In [ ]:
df.head()

In [ ]:
for i, row in tqdm.tqdm(df.iterrows()):

    embeddings = await get_embedding([row.text])

    df.loc[i, "text_embedding"] = str(embeddings[0].embedding)


In [ ]:
df.tail()

In [ ]:
df.to_csv("indexing/data/to_upsert/autocomplete/question_EMBED.csv", index=None)

In [ ]:
df.to_csv("indexing/data/to_upsert/autocomplete/eak_admin_ch_de_fr_tags_EMBED.csv", index=None)

# EMBEDDING

In [16]:
MAX_INPUT_TOKENS = 8191

In [17]:
import tiktoken
tokenizer = tiktoken.get_encoding("cl100k_base")

In [18]:
import sys
import os

# Add the src directory to the system path
sys.path.append(os.path.abspath(os.path.join('..', '..', 'src', 'copilot', 'app')))

# Now you can import the function
from utils.embedding import get_embedding

## WITHOUT CHUNKING

In [8]:
df["text_embedding"] = None
df["tags_embedding"] = None
df["subtopics_embedding"] = None
df["summary_embedding"] = None
df["hyq_embedding"] = None
df["hyq_declarative_embedding"] = None

In [ ]:
for i, row in tqdm.tqdm(df.iterrows()):

    try:
        tokens = tokenizer.encode(row.text)
        if len(tokens) > MAX_INPUT_TOKENS:
            truncated_text = tokenizer.decode(tokens[:MAX_INPUT_TOKENS])
            embeddings = await get_embedding(truncated_text)
        else:
            embeddings = await get_embedding(row.text)            
        tags_embedding = await get_embedding(row.tags)
        summary_embedding = await get_embedding(row.summary)
        hyq_embedding = await get_embedding(row.hyq)
        hyq_declarative_embedding = await get_embedding(row.hyq_declarative)
        subtopics_embedding = await get_embedding(row.subtopics)
    except Exception as e:
        embeddings = None

    df.loc[i, "text_embedding"] = str(embeddings) if embeddings else None
    df.loc[i, "summary_embedding"] = str(summary_embedding)
    df.loc[i, "hyq_embedding"] = str(hyq_embedding)
    df.loc[i, "hyq_declarative_embedding"] = str(hyq_declarative_embedding)
    df.loc[i, "subtopics_embedding"] = str(subtopics_embedding)
    df.loc[i, "tags_embedding"] = str(tags_embedding)


2025-03-10 15:06:50,537 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-03-10 15:06:50,540 - utils.embedding - INFO - Embedding successfull with model: text-embedding-3-small
2025-03-10 15:06:51,125 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-03-10 15:06:51,129 - utils.embedding - INFO - Embedding successfull with model: text-embedding-3-small
2025-03-10 15:06:54,168 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-03-10 15:06:54,171 - utils.embedding - INFO - Embedding successfull with model: text-embedding-3-small
2025-03-10 15:06:54,804 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-03-10 15:06:54,806 - utils.embedding - INFO - Embedding successfull with model: text-embedding-3-small
2025-03-10 15:06:55,114 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 2

## WITH CHUNKING

In [19]:
import math

def chunk_text_uniform(text: str, tokenizer, max_tokens: int = MAX_INPUT_TOKENS, overlap: int = 128):
    """
    Split text into evenly sized chunks, each up to max_tokens in length, with optional overlap.

    :param text: The input text to split.
    :param tokenizer: A tokenizer with encode/decode methods (e.g., from tiktoken or HuggingFace).
    :param max_tokens: Maximum tokens allowed in each chunk.
    :param overlap: Number of tokens to overlap between consecutive chunks.
    :return: A generator yielding each chunk as a string.
    """
    tokens = tokenizer.encode(text)
    n_tokens = len(tokens)
    
    # If the text is within the limit, just yield it as one chunk
    if n_tokens <= max_tokens:
        yield tokenizer.decode(tokens)
        return

    # 1) Decide how many chunks we need, ignoring overlap for a moment
    #    We subtract overlap to ensure we won't exceed max_tokens on each chunk
    chunk_count = math.ceil(n_tokens / (max_tokens - overlap))

    # 2) Compute an ideal chunk size so all chunks are about the same length
    #    (some chunks might be smaller if n_tokens isn't divisible)
    chunk_size = math.ceil(n_tokens / chunk_count)

    # 3) Generate chunks in a sliding window with overlap
    start = 0
    for i in range(chunk_count):
        end = min(start + chunk_size, n_tokens)
        chunk_tokens = tokens[start:end]
        yield tokenizer.decode(chunk_tokens)

        # Move start forward by chunk_size - overlap
        # so we get the desired overlap on the next chunk
        start += (chunk_size - overlap)
        if start >= n_tokens:
            break


In [24]:
async def build_chunked_df(df: pd.DataFrame) -> pd.DataFrame:
    new_rows = []

    for i, row in tqdm.tqdm(df.iterrows(), total=len(df)):
        # 1) Embed fields that are NOT chunked, only once
        try:
            tags_embedding = await get_embedding(row.tags)
            summary_embedding = await get_embedding(row.summary)
            hyq_embedding = await get_embedding(row.hyq)
            hyq_declarative_embedding = await get_embedding(row.hyq_declarative)
            subtopics_embedding = await get_embedding(row.subtopics)
        except Exception as e:
            # If embedding fails for these fields, skip or handle differently
            tags_embedding = summary_embedding = hyq_embedding = None
            hyq_declarative_embedding = subtopics_embedding = None

        # 2) Handle chunking of 'text'
        tokens = tokenizer.encode(row.text)
        if len(tokens) <= MAX_INPUT_TOKENS:
            # No need to chunk
            chunked_texts = [row.text]
        else:
            # Chunk into uniform segments
            chunked_texts = list(chunk_text_uniform(
                row.text,
                tokenizer
            ))
            
        for chunk in chunked_texts:
            try:
                chunk_embedding = await get_embedding(chunk)
            except:
                chunk_embedding = None

            # Create a new row for each chunk
            new_rows.append({
                **row.to_dict(),
                "text": chunk,
                "text_embedding": chunk_embedding,
                "tags_embedding": tags_embedding,
                "summary_embedding": summary_embedding,
                "hyq_embedding": hyq_embedding,
                "hyq_declarative_embedding": hyq_declarative_embedding,
                "subtopics_embedding": subtopics_embedding
            })

    # Convert the collected rows into a new DataFrame
    return pd.DataFrame(new_rows)

In [25]:
df_chunked = await build_chunked_df(df)

  0%|          | 0/64 [00:00<?, ?it/s]2025-03-10 16:35:55,860 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-03-10 16:35:55,866 - utils.embedding - INFO - Embedding successfull with model: text-embedding-3-small
2025-03-10 16:35:56,169 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-03-10 16:35:56,173 - utils.embedding - INFO - Embedding successfull with model: text-embedding-3-small
2025-03-10 16:35:56,523 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-03-10 16:35:56,528 - utils.embedding - INFO - Embedding successfull with model: text-embedding-3-small
2025-03-10 16:35:57,167 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-03-10 16:35:57,200 - utils.embedding - INFO - Embedding successfull with model: text-embedding-3-small
2025-03-10 16:35:57,817 - httpx - INFO - HTTP Request: POST https://ap

In [26]:
df. info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   url              64 non-null     object
 1   text             64 non-null     object
 2   language         64 non-null     object
 3   tags             64 non-null     object
 4   subtopics        64 non-null     object
 5   summary          64 non-null     object
 6   doctype          64 non-null     object
 7   organizations    64 non-null     object
 8   hyq              64 non-null     object
 9   hyq_declarative  64 non-null     object
dtypes: object(10)
memory usage: 5.1+ KB


In [27]:
df_chunked.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283 entries, 0 to 282
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   url                        283 non-null    object
 1   text                       283 non-null    object
 2   language                   283 non-null    object
 3   tags                       283 non-null    object
 4   subtopics                  283 non-null    object
 5   summary                    283 non-null    object
 6   doctype                    283 non-null    object
 7   organizations              283 non-null    object
 8   hyq                        283 non-null    object
 9   hyq_declarative            283 non-null    object
 10  text_embedding             283 non-null    object
 11  tags_embedding             283 non-null    object
 12  summary_embedding          283 non-null    object
 13  hyq_embedding              283 non-null    object
 14  hyq_declar

In [28]:
df_chunked.tail()

,url,text,language,tags,subtopics,summary,doctype,organizations,hyq,hyq_declarative,text_embedding,tags_embedding,summary_embedding,hyq_embedding,hyq_declarative_embedding,subtopics_embedding
278,https://sozialversicherungen.admin.ch/de/d/693...,EDI BSV | Nachtrag 14 zur Wegleitung über die ...,de,el_services,"Dokumente, Grundlagen_EL, NachtrÃ¤ge_zu_Weisungen",Der Nachtrag 14 zur Wegleitung über die Ergänz...,context_doc,OFAS,Welche neuen Bestimmungen enthält der Nachtrag...,"Ich möchte wissen, welche neuen Bestimmungen d...","[-2.0038100956298877e-06, 0.03218306601047516,...","[-0.02400936931371689, -0.015961579978466034, ...","[0.0034891481045633554, 0.0375317707657814, 0....","[-0.0049408902414143085, 0.021171314641833305,...","[-0.0012904723407700658, 0.01591029018163681, ...","[-0.037057019770145416, 0.059282295405864716, ..."
279,https://sozialversicherungen.admin.ch/de/d/693...,"von Personen, die nicht dauernd oder längere Z...",de,el_services,"Dokumente, Grundlagen_EL, NachtrÃ¤ge_zu_Weisungen",Der Nachtrag 14 zur Wegleitung über die Ergänz...,context_doc,OFAS,Welche neuen Bestimmungen enthält der Nachtrag...,"Ich möchte wissen, welche neuen Bestimmungen d...","[0.021957892924547195, 0.029770178720355034, 0...","[-0.02400936931371689, -0.015961579978466034, ...","[0.0034891481045633554, 0.0375317707657814, 0....","[-0.0049408902414143085, 0.021171314641833305,...","[-0.0012904723407700658, 0.01591029018163681, ...","[-0.037057019770145416, 0.059282295405864716, ..."
280,https://sozialversicherungen.admin.ch/de/d/693...,paar lebt mit seinen beiden Kindern (8 und 5 J...,de,el_services,"Dokumente, Grundlagen_EL, NachtrÃ¤ge_zu_Weisungen",Der Nachtrag 14 zur Wegleitung über die Ergänz...,context_doc,OFAS,Welche neuen Bestimmungen enthält der Nachtrag...,"Ich möchte wissen, welche neuen Bestimmungen d...","[0.01924232952296734, 0.033576346933841705, 0....","[-0.02400936931371689, -0.015961579978466034, ...","[0.0034891481045633554, 0.0375317707657814, 0....","[-0.0049408902414143085, 0.021171314641833305,...","[-0.0012904723407700658, 0.01591029018163681, ...","[-0.037057019770145416, 0.059282295405864716, ..."
281,https://sozialversicherungen.admin.ch/de/d/693...,barung hinsicht- lich der Höhe des Unterhaltes...,de,el_services,"Dokumente, Grundlagen_EL, NachtrÃ¤ge_zu_Weisungen",Der Nachtrag 14 zur Wegleitung über die Ergänz...,context_doc,OFAS,Welche neuen Bestimmungen enthält der Nachtrag...,"Ich möchte wissen, welche neuen Bestimmungen d...","[0.006381860468536615, 0.044872090220451355, 0...","[-0.02400936931371689, -0.015961579978466034, ...","[0.0034891481045633554, 0.0375317707657814, 0....","[-0.0049408902414143085, 0.021171314641833305,...","[-0.0012904723407700658, 0.01591029018163681, ...","[-0.037057019770145416, 0.059282295405864716, ..."
282,https://sozialversicherungen.admin.ch/de/d/184...,Nachtrag 4 zum Kreisschreiben über das Verfahr...,de,el_services,"Dokumente, Grundlagen_EL, NachtrÃ¤ge_zu_Weisungen",Der Nachtrag 4 zum Kreisschreiben über das Ver...,context_doc,OFAS,Welche Änderungen treten mit dem Nachtrag 4 zu...,"Ich möchte wissen, welche Änderungen der Nacht...","[-0.002204330638051033, 0.024825917556881905, ...","[-0.023945046588778496, -0.01598438248038292, ...","[-0.006297260522842407, 0.03352046757936478, 0...","[-0.015947749838232994, 0.018356582149863243, ...","[-0.00039202201878651977, 0.012620405294001102...","[-0.0370328463613987, 0.05927043408155441, 0.0..."


In [ ]:
df.head()

In [32]:
df_chunked.to_csv("data/zas_eak_copilot/ofas/ofas_EMBED.csv", index=None)

In [ ]:
df = pd.read_csv("indexing/data/zas_eak_copilot/akis/akis_EMBED_2.csv")

In [ ]:
df.tail()